In [1]:
#imports
import requests
import pandas as pd
import datetime as dt
from google.cloud import bigquery
from google.cloud import storage
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
import numpy as np
import meteostat as ms
import os
import io
import json

In [2]:
CREDS = "C:/Users/tkkim/gcp_keys/capstone-team51-366963bafc54.json"
storage_client = storage.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
bq_client = bigquery.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
#client = bigquery.Client(project='capstone-team51')


In [3]:
geolocator = Nominatim(user_agent="test_tk")

In [4]:
bucket = storage_client.get_bucket('capstone-team51-data')

In [121]:
def get_commarea_env(df_, commarea):
    dataframe = df_.copy()
    dataframe['community_area']=''
    for row in dataframe.index:
        point = Point(dataframe.loc[row,'longitude'],dataframe.loc[row,'latitude'])
        #print(point.xy)
        for feature in commarea['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                dataframe.loc[row, 'community_area'] = feature['properties']['area_numbe']
                
    return dataframe

In [6]:
def getpoint(address):
    loc = geolocator.geocode(address)
    latlong = [loc.latitude, loc.longitude]
    point = Point(latlong[1], latlong[0])   
    return point

In [7]:
def findarea(point, commarea):    
    for feature in commarea['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
           return feature['properties']['area_numbe']
        else:
            continue
        

In [8]:
def load_df_gcs(prefix, filename):
    df = pd.DataFrame()
    for blob in storage_client.list_blobs('capstone-team51-data', prefix=prefix, delimiter='/'):
        #print(blob.name)
        if filename in blob.name:
            #print(blob.name)
            data = blob.download_as_bytes()
            smalldf = pd.read_csv(io.BytesIO(data))
            #print(smalldf)
            df = pd.concat([df, smalldf])
    return df

In [30]:
base_df = load_df_gcs('raw_crimes/', 'chunk')
three11_df = load_df_gcs('raw_311/', 'chunk')
env_df = load_df_gcs('raw_environmental/', 'chunk')
weather_df = load_df_gcs('raw_weather/', 'data')
offenders_df = load_df_gcs('supporting/off_commarea/', 'csv')

In [31]:
#stations_blob = bucket.blob('supporting_data/Police_Stations_20240120.csv').download_as_bytes()
stations_df = load_df_gcs('supporting/', 'Police_Stations_20240120.csv')

In [88]:
commarea = json.loads(bucket.blob('supporting/geojsons/Boundaries - Community Areas (current).geojson').download_as_string())

In [122]:
env_df_ca = get_commarea_env(env_df, commarea)

In [89]:
# dph env complaints

grouped_complaints_by_commarea = env_df.groupby('community_area').agg(total_complaints_in_ca=('complaint_id','count')).reset_index()

(array('d', [-87.65552929240022]), array('d', [41.917035884002566]))
(array('d', [-87.6943035754064]), array('d', [41.7726350742548]))
(array('d', [-87.62226362586607]), array('d', [41.89245644738031]))
(array('d', [-87.66062216016986]), array('d', [41.77597289860283]))
(array('d', [-87.66456137666425]), array('d', [41.88132153435288]))
(array('d', [-87.71989473766482]), array('d', [41.92467746779379]))
(array('d', [-87.65448693822991]), array('d', [41.919591750448426]))
(array('d', [-87.65291910670649]), array('d', [41.80683556082504]))
(array('d', [-87.68597865072671]), array('d', [41.91574453238172]))
(array('d', [-87.66850812645559]), array('d', [41.84676817855723]))
(array('d', [-87.70332069298372]), array('d', [41.92559760883941]))
(array('d', [-87.59839106098786]), array('d', [41.80973408310273]))
(array('d', [-87.59839106098786]), array('d', [41.80973408310273]))
(array('d', [-87.6587673705282]), array('d', [41.9780530838796]))
(array('d', [-87.6812518370052]), array('d', [41.8

In [93]:
point = Point(env_df.loc[0,'longitude'],env_df.loc[0,'latitude'])
#print(point.xy)
for feature in commarea['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
        print(feature['properties']['area_numbe'])

7


In [66]:
# 311 complaints
grouped_date = three11_df.groupby([three11_df['created_date'].dt.date,'community_area','sr_type']).agg(total_count=('sr_number','count')).reset_index()
pivoted = grouped_date.pivot_table(index=['created_date','community_area'],columns='sr_type',values='total_count',aggfunc='sum').fillna(0).reset_index()
pivoted['created_date'] = pd.to_datetime(pivoted['created_date'])
pivoted.rename(columns={'created_date':'date'}, inplace=True)

In [74]:
# merge offenders in
big_df = base_df.merge(offenders_df, how='left', on='community_area')
big_df['date'] = pd.to_datetime(big_df['date'])

In [82]:
grouped_complaints_by_commarea

,community_area,total_complaints_in_ca
0,9999,95


In [76]:
# merge dph env complaints
big_df = big_df.merge(grouped_complaints_by_commarea, how='left', left_on='community_area', right_on='community_area')

In [78]:
big_df.total_complaints_in_ca.sum()

0.0

In [72]:
big_df = pivoted.merge(pivoted,how='left', on=['date', 'community_area'])

In [73]:
big_df.head()

sr_type,date,community_area,Abandoned Vehicle Complaint_x,Aircraft Noise Complaint_x,Alley Light Out Complaint_x,Alley Pothole Complaint_x,Alley Sewer Inspection Request_x,Animal In Trap Complaint_x,Bee/Wasp Removal_x,Bicycle Request/Complaint_x,...,Viaduct Light Out Complaint_y,Vicious Animal Complaint_y,Wage Complaint_y,Water Lead Test Kit Request_y,Water Lead Test Visit Request_y,Water On Street Complaint_y,Water Quality Concern_y,Water in Basement Complaint_y,Wire Basket Request_y,Yard Waste Pick-Up Request_y
0,2023-12-31,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-12-31,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2023-12-31,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-12-31,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-12-31,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
prefix_file_dict = {'raw_crimes/': 'chunk', 
                    'raw_311/': 'chunk', 
                    'raw_environmental/': 'chunk', 
                    'raw_weather/': 'data'}

In [77]:
df_holder = []
for key, value in prefix_file_dict.items():
    smallholder = {}
    df = load_df_gcs(key, value)
    smallholder['name'] = key
    smallholder['df'] = df
    df_holder.append(smallholder)

raw_crimes/
raw_311/
raw_environmental/
raw_weather/


In [65]:
x = load_df_gcs('raw_weather/', 'data')